### GA RMSE

In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error

def roulette_wheel_selection(population, fitness_scores):
    """
    룰렛 휠 선택 방식으로 부모 개체를 선택
    Args:
        population: 현재 개체군 (가중치 배열)
        fitness_scores: 각 개체의 RMSE (낮을수록 좋음)
    Returns:
        선택된 부모 개체
    """
    fitness_list = []
    max_fitness = max(fitness_scores)
    min_fitness = min(fitness_scores)
    adjustment = (max_fitness - min_fitness) / 2 if max_fitness != min_fitness else 1  # 조정 값

    for m in range(len(fitness_scores)):
        # fitness 계산
        fitness = max_fitness - fitness_scores[m] + adjustment
        fitness_list.append(fitness)

    fitness_list = np.array(fitness_list)
    total_fitness = np.sum(fitness_list)

    # total_fitness가 0인 경우 예외 처리
    if total_fitness == 0:
        raise ValueError("Total fitness is 0, cannot calculate probabilities.")

    probabilities = fitness_list / total_fitness

    # 룰렛 휠 방식으로 부모 선택
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index]


def genetic_algorithm_with_elitism(model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03):
    """
    GA를 사용하여 최적의 가중치 찾기
    Args:
        model_predictions: (n_samples, models, time_window, 2) 형태의 모델 예측값
        true_values: (n_samples, time_window, 2) 형태의 실제값
        population_size: 초기 개체군 크기
        generations: 세대 수
        mutation_rate: 돌연변이 확률
    Returns:
        최적 가중치 배열
    """
    num_models = model_predictions.shape[1]  # 모델 수
    population = np.random.dirichlet(np.ones(num_models), size=population_size)  # 초기 가중치 개체군

    for generation in range(generations):
        fitness_scores = []

        # 현재 population의 fitness (RMSE) 계산
        for individual in population:
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            rmse = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))
            fitness_scores.append(rmse)
        
        fitness_scores = np.array(fitness_scores)

        # 새로운 population 생성
        new_population = []
        while len(new_population) < population_size:
            # 부모 선택
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)

            # 교차 연산
            crossover_point = np.random.randint(1, num_models)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

            # 돌연변이
            if np.random.rand() < mutation_rate:
                mutation_vector = np.random.normal(0, 0.1, size=num_models)
                child = np.clip(child + mutation_vector, 0, 1)

            # 정규화
            child = child / np.sum(child)
            new_population.append(child)
        
        new_population = np.array(new_population)

        # 엘리티즘 적용
        combined_population = np.vstack((population, new_population))
        combined_fitness = np.concatenate((fitness_scores, np.zeros(len(new_population))))

        for idx, individual in enumerate(new_population):
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            combined_fitness[len(fitness_scores) + idx] = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))

        sorted_indices = np.argsort(combined_fitness)
        population = combined_population[sorted_indices][:population_size]

        print(f"Generation {generation + 1}/{generations}, Best RMSE: {combined_fitness[sorted_indices][0]}")

    # 최적 개체 반환
    best_weights = population[0]
    print(f"Optimal Weights: {best_weights}")
    return best_weights


In [15]:
import os
import numpy as np
import pandas as pd

def load_and_process_validation_data(input_directory, month, models=5, time_window=48):
    model_predictions = []  # 각 모델의 예측값 저장

    for model_idx in range(1, models + 1):
        model_dir = os.path.join(input_directory, f"model{model_idx}")
        file_path = os.path.join(model_dir, f'val_month_{month}_model_{model_idx}_results.csv')

        # 🔍 파일 존재 여부 확인
        if not os.path.exists(file_path):
            print(f"파일이 존재하지 않습니다: {file_path}")
            continue

        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"CSV 파일을 읽는 중 오류 발생: {file_path}, 오류: {e}")
            continue

        # 🔍 컬럼 존재 여부 확인
        pred_u_col = f"Model {model_idx} Val Pred U"
        pred_v_col = f"Model {model_idx} Val Pred V"

        if pred_u_col not in df.columns or pred_v_col not in df.columns:
            print(f"예측 컬럼이 존재하지 않습니다: {pred_u_col}, {pred_v_col}")
            continue

        pred_u = df[pred_u_col].values
        pred_v = df[pred_v_col].values

        # True 값은 모델 1에서만 추출
        if model_idx == 1:
            if "True U" not in df.columns or "True V" not in df.columns:
                print(f"True 값 컬럼이 존재하지 않습니다.")
                continue

            true_u = df["True U"].values
            true_v = df["True V"].values

        # 🔍 데이터 길이 확인 및 예외 처리
        n_samples = len(pred_u) // time_window
        if n_samples == 0:
            print(f"{file_path}: 데이터가 부족합니다. (데이터 길이: {len(pred_u)})")
            continue

        pred_u = np.array(pred_u[:n_samples * time_window]).reshape(n_samples, time_window, 1)
        pred_v = np.array(pred_v[:n_samples * time_window]).reshape(n_samples, time_window, 1)
        model_predictions.append(np.concatenate([pred_u, pred_v], axis=2))  # (n_samples, 48, 2)

        # True 값 병합 (한 번만 실행)
        if model_idx == 1:
            true_u = np.array(true_u[:n_samples * time_window]).reshape(n_samples, time_window, 1)
            true_v = np.array(true_v[:n_samples * time_window]).reshape(n_samples, time_window, 1)
            true_values = np.concatenate([true_u, true_v], axis=2)  # (n_samples, 48, 2)

    # 🔍 예측값이 없는 경우 예외 처리
    if not model_predictions:
        print(f"{month}월: 모델 예측값이 없습니다.")
        return np.array([]), np.array([])

    # 🔍 모델 예측값 형태 변환: (n_samples, models, 48, 2)
    try:
        model_predictions = np.stack(model_predictions, axis=1)
    except ValueError as e:
        print(f"모델 예측값 병합 중 오류 발생: {e}")
        return np.array([]), np.array([])

    return model_predictions, true_values


In [16]:
# 새로운 블록: GA 실행 및 최적 가중치 찾기
input_directory = 'ga_s3'
optimal_weights_list=[]
for month in range(1, 13):
    try:

        model_predictions, true_values = load_and_process_validation_data(input_directory, month)
        print(model_predictions.shape, true_values.shape)
        if len(model_predictions) == 0 or len(true_values) == 0:
            print(f"{month}월: 데이터가 부족하여 건너뜁니다.")
            continue

        # GA 실행하여 최적 가중치 찾기
        optimal_weights = genetic_algorithm_with_elitism(
            model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03
        )
        optimal_weights_list.append(optimal_weights)
    
    except Exception as e:
        print(f"{month}월 처리 중 오류 발생: {e}")

(25, 5, 48, 2) (25, 48, 2)
Generation 1/100, Best RMSE: 2.2776612552401785
Generation 2/100, Best RMSE: 2.2766196785739576
Generation 3/100, Best RMSE: 2.2760045730339993
Generation 4/100, Best RMSE: 2.2754268155036628
Generation 5/100, Best RMSE: 2.2752889030020236
Generation 6/100, Best RMSE: 2.2752768751689527
Generation 7/100, Best RMSE: 2.2752690786487415
Generation 8/100, Best RMSE: 2.2752387204168287
Generation 9/100, Best RMSE: 2.275173768291438
Generation 10/100, Best RMSE: 2.2750043261809787
Generation 11/100, Best RMSE: 2.274799034176094
Generation 12/100, Best RMSE: 2.2747788565662983
Generation 13/100, Best RMSE: 2.2747728535766076
Generation 14/100, Best RMSE: 2.2747652134862117
Generation 15/100, Best RMSE: 2.2747587421148094
Generation 16/100, Best RMSE: 2.274710212113445
Generation 17/100, Best RMSE: 2.274710212113445
Generation 18/100, Best RMSE: 2.274710212113445
Generation 19/100, Best RMSE: 2.2747055744470415
Generation 20/100, Best RMSE: 2.2747054174303045
Generat

In [18]:
for i in optimal_weights_list:
    print(i)

[0.57055253 0.         0.01078756 0.16910843 0.24955148]
[0.89326787 0.01718011 0.08480656 0.         0.00474546]
[0.87158191 0.         0.01500284 0.         0.11341526]
[0.48293436 0.         0.         0.01971699 0.49734865]
[0.73229964 0.         0.         0.         0.26770036]
[0.29015045 0.19695908 0.34140963 0.         0.17148084]
[0.43500006 0.         0.         0.14708702 0.41791292]
[0.83995537 0.         0.         0.12841461 0.03163001]
[0.58545155 0.         0.27482755 0.         0.1397209 ]
[0.43205361 0.01455957 0.         0.         0.55338682]
[0.35697415 0.         0.         0.         0.64302585]
[0.8902205  0.         0.09566673 0.01411277 0.        ]


In [19]:
import os
import pandas as pd
import numpy as np

def load_and_process_test_data(input_directory, month, models=5, time_window=48):
    """
    Test 데이터를 로드하고, 모델 예측값과 True 값을 추출해 시간 단위로 이어붙이기

    Args:
        input_directory (str): 모델별 test 데이터가 있는 디렉토리
        month (int): 처리할 월
        models (int): 모델 개수 (default=5)
        time_window (int): 샘플당 시간 창 (default=48)

    Returns:
        model_predictions (np.ndarray): (n_samples, models, time_window, 2)
        true_values (np.ndarray): (n_samples, time_window, 2)
    """
    model_predictions = []  # 모델별 예측값 저장
    true_values = None  # True U, True V 저장

    for model_idx in range(1, models + 1):
        model_dir = os.path.join(input_directory, f"model{model_idx}")
        file_path = os.path.join(model_dir, f'test_month_{month}_model_{model_idx}_results.csv')

        # 파일 존재 여부 확인
        if not os.path.exists(file_path):
            print(f"Warning: {file_path} not found. Skipping model {model_idx}.")
            continue

        df = pd.read_csv(file_path)

        # NaN 값 확인
        if df.isnull().values.any():
            print(f"NaN found in Model {model_idx} test results for month {month}:")
            print(df.isnull().sum())

        # 모델 예측값 추출
        pred_u = df[f"Model {model_idx} Test Pred U"].values
        pred_v = df[f"Model {model_idx} Test Pred V"].values

        # 데이터 부족 시 예외 처리
        n_samples = len(pred_u) // time_window
        if n_samples == 0:
            print(f"Warning: Insufficient data for model {model_idx} in month {month}. Skipping...")
            continue

        pred_u = pred_u[:n_samples * time_window].reshape(n_samples, time_window, 1)
        pred_v = pred_v[:n_samples * time_window].reshape(n_samples, time_window, 1)
        model_data = np.concatenate([pred_u, pred_v], axis=2)  # (n_samples, 48, 2)

        # 모델별 예측값 저장
        model_predictions.append(model_data)

        # True U와 True V 추출 (첫 번째 모델 파일에서만 처리)
        if model_idx == 1:
            true_u = df["True U"].values[:n_samples * time_window].reshape(n_samples, time_window, 1)
            true_v = df["True V"].values[:n_samples * time_window].reshape(n_samples, time_window, 1)
            true_values = np.concatenate([true_u, true_v], axis=2)

    # 모델 예측값 형태 변환: (n_samples, models, 48, 2)
    if model_predictions:
        model_predictions = np.stack(model_predictions, axis=1)
    else:
        raise ValueError("No valid data found for any model.")

    # True Values가 정상적으로 로드되었는지 확인
    if true_values is None:
        raise ValueError("True values could not be extracted.")

    print(f"Processed Month {month} - Model Predictions Shape: {model_predictions.shape}")
    print(f"Processed Month {month} - True Values Shape: {true_values.shape}")

    return model_predictions, true_values


In [20]:
def main():
    input_directory = "./ga_s3"
    
    # 1월부터 12월까지 데이터 로드 및 출력
    for month in range(1, 13):
        try:
            model_predictions, true_values = load_and_process_test_data(input_directory, month)
            print(f"\nMonth {month} - Model Predictions Shape: {model_predictions.shape}")
            print(f"Month {month} - True Values Shape: {true_values.shape}")
            
            # 추가적인 데이터 분석 또는 모델 평가 수행 가능
            # 예시: 전체 RMSE 계산
            mse = np.mean((model_predictions.mean(axis=1) - true_values) ** 2)
            rmse = np.sqrt(mse)
            print(f"Month {month} - RMSE: {rmse:.4f}")

        except ValueError as e:
            print(f"Skipping month {month} due to error: {e}")

if __name__ == "__main__":
    main()


Processed Month 1 - Model Predictions Shape: (26, 5, 48, 2)
Processed Month 1 - True Values Shape: (26, 48, 2)

Month 1 - Model Predictions Shape: (26, 5, 48, 2)
Month 1 - True Values Shape: (26, 48, 2)
Month 1 - RMSE: 2.0867
Processed Month 2 - Model Predictions Shape: (25, 5, 48, 2)
Processed Month 2 - True Values Shape: (25, 48, 2)

Month 2 - Model Predictions Shape: (25, 5, 48, 2)
Month 2 - True Values Shape: (25, 48, 2)
Month 2 - RMSE: 2.2926
Processed Month 3 - Model Predictions Shape: (28, 5, 48, 2)
Processed Month 3 - True Values Shape: (28, 48, 2)

Month 3 - Model Predictions Shape: (28, 5, 48, 2)
Month 3 - True Values Shape: (28, 48, 2)
Month 3 - RMSE: 2.3402
Processed Month 4 - Model Predictions Shape: (20, 5, 48, 2)
Processed Month 4 - True Values Shape: (20, 48, 2)

Month 4 - Model Predictions Shape: (20, 5, 48, 2)
Month 4 - True Values Shape: (20, 48, 2)
Month 4 - RMSE: 2.5049
Processed Month 5 - Model Predictions Shape: (18, 5, 48, 2)
Processed Month 5 - True Values Shap

In [21]:
from sklearn.metrics import mean_squared_error
import numpy as np

def calculate_ensemble_rmse(model_predictions, true_values, weights, output_steps):
    """
    최적 가중치를 사용하여 앙상블 예측값을 저장하고 나중에 RMSE를 계산

    Args:
        model_predictions (np.ndarray): 모델 예측값 (n_samples, models, time_window, 2)
        true_values (np.ndarray): 실제 값 (n_samples, time_window, 2)
        weights (np.ndarray): 최적 가중치 (models,)
        output_steps (int): 예측 타임스텝 수

    Returns:
        results_dict (dict): 예측값과 실제값 저장 딕셔너리
        overall_rmse (float): 전체 평균 RMSE
    """

    print(f"NaN in model_predictions: {np.isnan(model_predictions).any()}")
    print(f"NaN in true_values: {np.isnan(true_values).any()}")

    # 입력 데이터 형태 검증
    if model_predictions.ndim != 4 or true_values.ndim != 3:
        raise ValueError("입력 데이터의 형태가 올바르지 않습니다. "
                         "model_predictions는 (n_samples, models, time_window, 2) 형태여야 하고, "
                         "true_values는 (n_samples, time_window, 2) 형태여야 합니다.")

    n_samples, n_models, time_window, _ = model_predictions.shape

    # 디버깅 출력
    print(f"model_predictions.shape: {model_predictions.shape}")
    print(f"true_values.shape: {true_values.shape}")
    print(f"weights.shape: {weights.shape}")
    print(f"weights: {weights}")
    print(f"time_window: {time_window}, output_steps: {output_steps}")

    # weights 크기 검증
    if len(weights) != n_models:
        raise ValueError(f"weights 길이({len(weights)})가 모델 개수({n_models})와 일치하지 않습니다.")

    # weights 합 검증
    if not np.isclose(np.sum(weights), 1.0):
        raise ValueError(f"weights의 합({np.sum(weights)})이 1이 아닙니다. 앙상블 결과가 왜곡될 수 있습니다.")

    # 앙상블 예측값 계산 (가중치 적용)
    ensemble_predictions = np.tensordot(model_predictions, weights, axes=(1, 0))  # (n_samples, time_window, 2)

    # 예측값과 실제값을 저장할 딕셔너리 초기화
    results_dict = {i: {'predictions': [], 'true_values': []} for i in range(time_window)}

    # 예측값과 실제값 저장
    for i in range(time_window):  # 전체 time_window만큼 반복
        results_dict[i]['predictions'].extend(ensemble_predictions[:, i, :].tolist())
        results_dict[i]['true_values'].extend(true_values[:, i, :].tolist())

    # 전체 RMSE 계산
    all_predictions = np.concatenate(
        [np.array(results_dict[i]['predictions']) for i in range(time_window)], axis=0
    )
    all_true_values = np.concatenate(
        [np.array(results_dict[i]['true_values']) for i in range(time_window)], axis=0
    )
    mse = mean_squared_error(all_true_values, all_predictions, multioutput="uniform_average")
    overall_rmse = np.sqrt(mse)


    return results_dict, round(overall_rmse, 4)


### Uniform RMSE

In [22]:
def calculate_uniform_ensemble_rmse(model_predictions, true_values):
    n_samples, n_models, time_window, n_features = model_predictions.shape

    # 동일 가중치 설정
    weights = np.array([1.0 / n_models] * n_models).reshape(1, -1, 1, 1)
    ensemble_predictions = np.sum(model_predictions * weights, axis=1)


    print(f"Ensemble Predictions Shape: {ensemble_predictions.shape}")

    results_dict = {i: [] for i in range(time_window)}

    for i in range(time_window):
        predictions_sample = ensemble_predictions[:, i, :]
        true_values_sample = true_values[:, i, :]

        mse = mean_squared_error(true_values_sample, predictions_sample, multioutput='uniform_average')
        rmse = np.sqrt(mse)

        results_dict[i].append(rmse)

    all_predictions = ensemble_predictions.reshape(-1, n_features)
    all_true_values = true_values.reshape(-1, n_features)
    print(all_true_values.shape)
    overall_mse = mean_squared_error(all_true_values, all_predictions, multioutput='uniform_average')
    overall_rmse = np.sqrt(overall_mse)

    return results_dict, overall_rmse


def calculate_groupwise_rmse(all_results):
    time_window = len(all_results[0])
    average_rmse = {i: [] for i in range(time_window)}

    for month_results in all_results:
        for key, rmse_list in month_results.items():
            average_rmse[key].extend(rmse_list)

    groupwise_rmse = {key: np.mean(rmse_list) for key, rmse_list in average_rmse.items()}
    overall_rmse = np.mean(list(groupwise_rmse.values()))

    return groupwise_rmse, overall_rmse


def main():
    input_directory = "./ga_s3"
    all_results = []
    overall_rmse_list = []

    for month in range(1, 13):
        model_predictions, true_values = load_and_process_test_data(input_directory, month)
        print(model_predictions.shape)
        print
        results_dict, overall_rmse = calculate_uniform_ensemble_rmse(model_predictions, true_values)

        all_results.append(results_dict)
        overall_rmse_list.append(overall_rmse)

    groupwise_rmse, overall_rmse = calculate_groupwise_rmse(all_results)

    print("\n시간별 평균 RMSE:")
    for time, rmse in groupwise_rmse.items():
        print(f"시간 {time}: {rmse}")

    print("\n전체 평균 RMSE:")
    print(f"uniform 월별 전체 평균 RMSE: {np.mean(overall_rmse_list)}")
    print(f"uniform 그룹별 전체 평균 RMSE: {overall_rmse}")


main()


Processed Month 1 - Model Predictions Shape: (26, 5, 48, 2)
Processed Month 1 - True Values Shape: (26, 48, 2)
(26, 5, 48, 2)
Ensemble Predictions Shape: (26, 48, 2)
(1248, 2)
Processed Month 2 - Model Predictions Shape: (25, 5, 48, 2)
Processed Month 2 - True Values Shape: (25, 48, 2)
(25, 5, 48, 2)
Ensemble Predictions Shape: (25, 48, 2)
(1200, 2)
Processed Month 3 - Model Predictions Shape: (28, 5, 48, 2)
Processed Month 3 - True Values Shape: (28, 48, 2)
(28, 5, 48, 2)
Ensemble Predictions Shape: (28, 48, 2)
(1344, 2)
Processed Month 4 - Model Predictions Shape: (20, 5, 48, 2)
Processed Month 4 - True Values Shape: (20, 48, 2)
(20, 5, 48, 2)
Ensemble Predictions Shape: (20, 48, 2)
(960, 2)
Processed Month 5 - Model Predictions Shape: (18, 5, 48, 2)
Processed Month 5 - True Values Shape: (18, 48, 2)
(18, 5, 48, 2)
Ensemble Predictions Shape: (18, 48, 2)
(864, 2)
Processed Month 6 - Model Predictions Shape: (18, 5, 48, 2)
Processed Month 6 - True Values Shape: (18, 48, 2)
(18, 5, 48,

In [23]:
def calculate_single_model_rmse(model_predictions, true_values, model_index=4):
    n_samples, n_models, time_window, n_features = model_predictions.shape

    # 특정 모델의 예측값만 선택
    selected_model_predictions = model_predictions[:, model_index, :, :]
    print(f"Selected Model {model_index} Predictions Shape: {selected_model_predictions.shape}")

    results_dict = {i: [] for i in range(time_window)}

    for i in range(time_window):
        predictions_sample = selected_model_predictions[:, i, :]
        true_values_sample = true_values[:, i, :]

        mse = mean_squared_error(true_values_sample, predictions_sample, multioutput='uniform_average')
        rmse = np.sqrt(mse)

        results_dict[i].append(rmse)

    all_predictions = selected_model_predictions.reshape(-1, n_features)
    all_true_values = true_values.reshape(-1, n_features)
    
    overall_mse = mean_squared_error(all_true_values, all_predictions, multioutput='uniform_average')
    overall_rmse = np.sqrt(overall_mse)

    return results_dict, overall_rmse


def main():
    input_directory = "./ga_s3"
    all_results = []
    overall_rmse_list = []

    for month in range(1, 13):
        model_predictions, true_values = load_and_process_test_data(input_directory, month)
        print(f"Month {month} - Model Predictions Shape: {model_predictions.shape}")

        # 모델 인덱스 4의 RMSE 계산
        results_dict, overall_rmse = calculate_single_model_rmse(model_predictions, true_values, 4)

        all_results.append(results_dict)
        overall_rmse_list.append(overall_rmse)

    groupwise_rmse, overall_rmse = calculate_groupwise_rmse(all_results)

    print("\n시간별 평균 RMSE (Model 4):")
    for time, rmse in groupwise_rmse.items():
        print(f"시간 {time}: {rmse}")

    print("\n전체 평균 RMSE (Model 4):")
    print(f"Model 4 월별 전체 평균 RMSE: {np.mean(overall_rmse_list)}")
    print(f"Model 4 그룹별 전체 평균 RMSE: {overall_rmse}")

main()


Processed Month 1 - Model Predictions Shape: (26, 5, 48, 2)
Processed Month 1 - True Values Shape: (26, 48, 2)
Month 1 - Model Predictions Shape: (26, 5, 48, 2)
Selected Model 4 Predictions Shape: (26, 48, 2)
Processed Month 2 - Model Predictions Shape: (25, 5, 48, 2)
Processed Month 2 - True Values Shape: (25, 48, 2)
Month 2 - Model Predictions Shape: (25, 5, 48, 2)
Selected Model 4 Predictions Shape: (25, 48, 2)
Processed Month 3 - Model Predictions Shape: (28, 5, 48, 2)
Processed Month 3 - True Values Shape: (28, 48, 2)
Month 3 - Model Predictions Shape: (28, 5, 48, 2)
Selected Model 4 Predictions Shape: (28, 48, 2)
Processed Month 4 - Model Predictions Shape: (20, 5, 48, 2)
Processed Month 4 - True Values Shape: (20, 48, 2)
Month 4 - Model Predictions Shape: (20, 5, 48, 2)
Selected Model 4 Predictions Shape: (20, 48, 2)
Processed Month 5 - Model Predictions Shape: (18, 5, 48, 2)
Processed Month 5 - True Values Shape: (18, 48, 2)
Month 5 - Model Predictions Shape: (18, 5, 48, 2)
Sel

In [27]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 기존 optimal_weights_list 사용
def calculate_ga_ensemble_rmse(model_predictions, true_values, weights):
    n_samples, n_models, time_window, n_features = model_predictions.shape
    print(f"Input Model Predictions Shape: {model_predictions.shape}")

    # GA 기반 가중치를 적용한 앙상블 예측값 계산 (n_samples 적용)
    weights = np.array(weights).reshape(1, -1, 1, 1)  # 모델 수 차원과 맞춤
    ensemble_predictions = np.sum(model_predictions * weights, axis=1)

    print(f"Ensemble Predictions Shape after applying weights: {ensemble_predictions.shape}")

    results_dict = {i: [] for i in range(time_window)}

    for i in range(time_window):
        predictions_sample = ensemble_predictions[:, i, :]
        true_values_sample = true_values[:, i, :]

        # 예측값과 실제값의 차원 확인
        if predictions_sample.shape != true_values_sample.shape:
            raise ValueError(f"Shape mismatch: {predictions_sample.shape} vs {true_values_sample.shape}")

        mse = mean_squared_error(true_values_sample, predictions_sample, multioutput='uniform_average')
        rmse = np.sqrt(mse)

        results_dict[i].append(rmse)

    all_predictions = ensemble_predictions.reshape(n_samples * time_window, n_features)
    all_true_values = true_values.reshape(n_samples * time_window, n_features)

    overall_mse = mean_squared_error(all_true_values, all_predictions, multioutput='uniform_average')
    overall_rmse = np.sqrt(overall_mse)

    return results_dict, overall_rmse


def calculate_groupwise_rmse(all_results):
    time_window = len(all_results[0])
    average_rmse = {i: [] for i in range(time_window)}

    for month_results in all_results:
        for key, rmse_list in month_results.items():
            average_rmse[key].extend(rmse_list)

    groupwise_rmse = {key: np.mean(rmse_list) for key, rmse_list in average_rmse.items()}
    overall_rmse = np.mean(list(groupwise_rmse.values()))

    return groupwise_rmse, overall_rmse


def main():
    input_directory = "./ga_s3"
    all_results = []
    overall_rmse_list = []

    for month in range(1, 13):
        print(f"\nProcessing month {month} data...")
        model_predictions, true_values = load_and_process_test_data(input_directory, month)

        # 월별 최적 가중치 적용
        month_weights = optimal_weights_list[month - 1]
        print(f"Applying weights for month {month}: {month_weights}")

        results_dict, overall_rmse = calculate_ga_ensemble_rmse(model_predictions, true_values, month_weights)

        all_results.append(results_dict)
        overall_rmse_list.append(overall_rmse)

    groupwise_rmse, overall_rmse = calculate_groupwise_rmse(all_results)

    print("\n시간별 평균 RMSE:")
    for time, rmse in groupwise_rmse.items():
        print(f"시간 {time}: {rmse}")

    print("\n전체 평균 RMSE:")
    print(f"GA 월별 전체 평균 RMSE: {np.mean(overall_rmse_list)}")
    print(f"GA 그룹별 전체 평균 RMSE: {overall_rmse}")


if __name__ == "__main__":
    main()



Processing month 1 data...
Processed Month 1 - Model Predictions Shape: (26, 5, 48, 2)
Processed Month 1 - True Values Shape: (26, 48, 2)
Applying weights for month 1: [0.57055253 0.         0.01078756 0.16910843 0.24955148]
Input Model Predictions Shape: (26, 5, 48, 2)
Ensemble Predictions Shape after applying weights: (26, 48, 2)

Processing month 2 data...
Processed Month 2 - Model Predictions Shape: (25, 5, 48, 2)
Processed Month 2 - True Values Shape: (25, 48, 2)
Applying weights for month 2: [0.89326787 0.01718011 0.08480656 0.         0.00474546]
Input Model Predictions Shape: (25, 5, 48, 2)
Ensemble Predictions Shape after applying weights: (25, 48, 2)

Processing month 3 data...
Processed Month 3 - Model Predictions Shape: (28, 5, 48, 2)
Processed Month 3 - True Values Shape: (28, 48, 2)
Applying weights for month 3: [0.87158191 0.         0.01500284 0.         0.11341526]
Input Model Predictions Shape: (28, 5, 48, 2)
Ensemble Predictions Shape after applying weights: (28, 48

In [25]:
from sklearn.metrics import mean_squared_error
import numpy as np

overall_rmse_list = []
uniform_overall_rmse_list = []
individual_model_rmse = []  # 월별 개별 모델 RMSE 저장 리스트
timewise_model_rmse = []  # 시간별 개별 모델 RMSE 저장 리스트
comparison_results = []  # 구간별 평균 RMSE 비교 결과 저장 리스트

for month in range(1, 13):  # 1월부터 12월까지 반복
    try:
        # 테스트 데이터 로드 및 처리
        model_predictions, true_values = load_and_process_test_data(input_directory, month)

        # 월별 optimal_weights 확인
        if isinstance(optimal_weights_list, list):
            month_weights = optimal_weights_list[month - 1]
        else:
            raise ValueError("optimal_weights_list가 월별 가중치를 포함한 리스트가 아닙니다.")

        print(f"{month}월 가중치: {month_weights}")

        # 월별 개별 모델 RMSE 계산
        n_models = model_predictions.shape[1]
        time_window = model_predictions.shape[2]
        monthly_model_rmse = []  # 해당 월의 각 모델 RMSE 저장
        monthly_timewise_rmse = {t: [] for t in range(time_window)}  # 시간별 RMSE 저장

        for model_idx in range(n_models):
            model_prediction = model_predictions[:, model_idx, :, :]  # 해당 모델의 예측값 (n_samples, time_window, 2)
            
            # 월별 RMSE
            mse = mean_squared_error(true_values.reshape(-1, 2), model_prediction.reshape(-1, 2), multioutput="uniform_average")
            rmse = np.sqrt(mse)
            monthly_model_rmse.append(rmse)

            # 시간별 RMSE
            for t in range(time_window):
                time_mse = mean_squared_error(true_values[:, t, :], model_prediction[:, t, :], multioutput="uniform_average")
                time_rmse = np.sqrt(time_mse)
                monthly_timewise_rmse[t].append(time_rmse)

        individual_model_rmse.append(monthly_model_rmse)
        timewise_model_rmse.append(monthly_timewise_rmse)

        print(f"{month}월 개별 모델 RMSE: {monthly_model_rmse}")

        # 0~24, 25~47 시간 구간별 개별 모델 평균 RMSE 계산
        avg_rmse_0_24 = [np.mean([monthly_timewise_rmse[t][model_idx] for t in range(0, 25)]) for model_idx in range(n_models)]
        avg_rmse_25_47 = [np.mean([monthly_timewise_rmse[t][model_idx] for t in range(25, 48)]) for model_idx in range(n_models)]

        print(f"{month}월 개별 모델 0~24 시간 평균 RMSE: {avg_rmse_0_24}")
        print(f"{month}월 개별 모델 25~47 시간 평균 RMSE: {avg_rmse_25_47}")

        # 앙상블 RMSE 계산
        ensemble_rmse, overall_rmse = calculate_ensemble_rmse(model_predictions, true_values, month_weights, 48)
        print(f"{month}월 예측값: {model_predictions.shape}, 실제값: {true_values.shape}, 가중치: {month_weights}")

        # 0~24, 25~47 시간 구간별 앙상블 평균 RMSE 계산
        ensemble_predictions = np.tensordot(model_predictions, month_weights, axes=(1, 0))  # (n_samples, time_window, 2)
        avg_ensemble_rmse_0_24 = np.mean([
            np.sqrt(mean_squared_error(true_values[:, t, :], ensemble_predictions[:, t, :], multioutput="uniform_average"))
            for t in range(0, 25)
        ])
        avg_ensemble_rmse_25_47 = np.mean([
            np.sqrt(mean_squared_error(true_values[:, t, :], ensemble_predictions[:, t, :], multioutput="uniform_average"))
            for t in range(25, 48)
        ])

        print(f"{month}월 앙상블 0~24 시간 평균 RMSE: {avg_ensemble_rmse_0_24}")
        print(f"{month}월 앙상블 25~47 시간 평균 RMSE: {avg_ensemble_rmse_25_47}")

        # 결과 비교 저장
        comparison_results.append({
            "month": month,
            "model_avg_rmse_0_24": avg_rmse_0_24,
            "model_avg_rmse_25_47": avg_rmse_25_47,
            "ensemble_avg_rmse_0_24": avg_ensemble_rmse_0_24,
            "ensemble_avg_rmse_25_47": avg_ensemble_rmse_25_47
        })

        overall_rmse_list.append(overall_rmse)

    except Exception as e:
        print(f"{month}월 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

# 최종 평균 RMSE 출력
if overall_rmse_list:
    print("\n전체 월 평균 앙상블 RMSE: ", np.mean(overall_rmse_list))
else:
    print("\n앙상블 RMSE 결과가 없습니다.")

# 구간별 결과 출력
print("\n구간별 평균 RMSE 비교 결과:")
for result in comparison_results:
    print(f"{result['month']}월:")
    print(f"  모델 0~24 시간 평균 RMSE: {result['model_avg_rmse_0_24']}")
    print(f"  모델 25~47 시간 평균 RMSE: {result['model_avg_rmse_25_47']}")
    print(f"  앙상블 0~24 시간 평균 RMSE: {result['ensemble_avg_rmse_0_24']:.4f}")
    print(f"  앙상블 25~47 시간 평균 RMSE: {result['ensemble_avg_rmse_25_47']:.4f}")


Processed Month 1 - Model Predictions Shape: (26, 5, 48, 2)
Processed Month 1 - True Values Shape: (26, 48, 2)
1월 가중치: [0.57055253 0.         0.01078756 0.16910843 0.24955148]
1월 개별 모델 RMSE: [2.102773022886727, 2.5728350521417958, 2.140547987162377, 2.8394349124644007, 2.1355950177896177]
1월 개별 모델 0~24 시간 평균 RMSE: [1.9249162191020355, 2.3532967377659455, 2.011720749118762, 2.71549660640836, 1.9637904820177168]
1월 개별 모델 25~47 시간 평균 RMSE: [2.261869064416568, 2.745342605296209, 2.249991653130403, 2.9500569506062173, 2.2850251502750125]
NaN in model_predictions: False
NaN in true_values: False
model_predictions.shape: (26, 5, 48, 2)
true_values.shape: (26, 48, 2)
weights.shape: (5,)
weights: [0.57055253 0.         0.01078756 0.16910843 0.24955148]
time_window: 48, output_steps: 48
1월 예측값: (26, 5, 48, 2), 실제값: (26, 48, 2), 가중치: [0.57055253 0.         0.01078756 0.16910843 0.24955148]
1월 앙상블 0~24 시간 평균 RMSE: 1.8636253177638589
1월 앙상블 25~47 시간 평균 RMSE: 2.1854852436043437
Processed Month 2 - M

### GA 검증 실험

In [25]:
def calculate_rmse(model_predictions, true_values, individual):
    """ 주어진 개체에 대해 앙상블 예측값을 계산하고 RMSE를 반환 """
    ensemble_prediction = np.sum([weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)], axis=0)
    rmse = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))
    return rmse

In [60]:
def genetic_algorithm_with_elitism(model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03, seed=None):
    """
    GA를 사용하여 최적의 가중치 찾기
    Args:
        model_predictions: (n_samples, models, time_window, 2) 형태의 모델 예측값
        true_values: (n_samples, time_window, 2) 형태의 실제값
        population_size: 초기 개체군 크기
        generations: 세대 수
        mutation_rate: 돌연변이 확률
        seed: 난수 시드를 고정하기 위한 값
    Returns:
        최적 가중치 배열
    """
    num_models = model_predictions.shape[1]  # 모델 수
    time_window = model_predictions.shape[2]  # 시간 단위
    
    # 초기 population 생성
    population = np.random.dirichlet(np.ones(num_models), size=population_size)  # 초기 가중치 개체군

    for generation in range(generations):
        fitness_scores = []

        # 현재 population의 fitness (RMSE) 계산
        for individual in population:
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            rmse = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))
            fitness_scores.append(rmse)
        
        fitness_scores = np.array(fitness_scores)

        # 새로운 population 생성
        new_population = []
        while len(new_population) < population_size:
            # 부모 선택
            np.random.seed(seed + generation)  # 세대마다 다른 시드 적용
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)

            # 교차 연산
            np.random.seed(seed + generation + 1)  # 세대마다 다른 시드 적용
            crossover_point = np.random.randint(1, num_models)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

            # 돌연변이
            if np.random.rand() < mutation_rate:
                np.random.seed(seed + generation + 2)  # 세대마다 다른 시드 적용
                mutation_vector = np.random.normal(0, 0.1, size=num_models)
                child = np.clip(child + mutation_vector, 0, 1)

            # 정규화
            child = child / np.sum(child)
            new_population.append(child)
        
        new_population = np.array(new_population)

        # 엘리티즘 적용: 상위 50% 개체 선택
        combined_population = np.vstack((population, new_population))
        combined_fitness = np.concatenate((fitness_scores, np.zeros(len(new_population))))

        for idx, individual in enumerate(new_population):
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            combined_fitness[len(fitness_scores) + idx] = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))

        sorted_indices = np.argsort(combined_fitness)
        population = combined_population[sorted_indices][:population_size]

        print(f"Generation {generation + 1}/{generations}, Best RMSE: {combined_fitness[sorted_indices][0]}")

    # 최적 개체 반환
    best_weights = population[0]
    print(f"Optimal Weights: {best_weights}")
    return best_weights


In [61]:
import time
def run_ga_experiment(model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03, seed=None):
    """
    유전 알고리즘 실험을 한 번 실행하고, GA 계산 시간을 측정하여 최적 가중치를 반환합니다.
    """
    np.random.seed(seed)  # seed 설정
    
    start_time = time.time()

    optimal_weights = genetic_algorithm_with_elitism(model_predictions, true_values, 
                                                     population_size=population_size, 
                                                     generations=generations, mutation_rate=mutation_rate, seed=seed)

    ga_time = time.time() - start_time
    
    # 앙상블 예측
    ensemble_prediction = np.sum([weight * model_predictions[:, i, :, :] for i, weight in enumerate(optimal_weights)], axis=0)
    
    # 3D 배열을 2D 배열로 변환 (n_samples, time_window * 2)
    ensemble_prediction_2d = ensemble_prediction.reshape(-1, ensemble_prediction.shape[1] * ensemble_prediction.shape[2])
    true_values_2d = true_values.reshape(-1, true_values.shape[1] * true_values.shape[2])

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(true_values_2d, ensemble_prediction_2d))

    return optimal_weights, ga_time



In [62]:
from sklearn.metrics import mean_squared_error
import numpy as np

def calculate_ensemble_rmse2(model_predictions, true_values, weights):
    """
    최적 가중치를 사용하여 앙상블 RMSE를 계산

    Args:
        model_predictions (np.ndarray): 모델 예측값 (n_samples, models, time_window, 2)
        true_values (np.ndarray): 실제 값 (n_samples, time_window, 2)
        weights (np.ndarray): 최적 가중치 (models,)

    Returns:
        ensemble_rmse_dict (dict): 시간별 RMSE 딕셔너리
        overall_rmse (float): 전체 평균 RMSE
    """
    n_samples, n_models, time_window, _ = model_predictions.shape
    
    # 앙상블 예측값 계산 (가중치 적용)
    ensemble_predictions = np.tensordot(model_predictions, weights, axes=(1, 0))  # (n_samples, time_window, 2)

    # 시간별 RMSE를 저장할 딕셔너리
    ensemble_rmse_dict = {}
    
    for time_step in range(time_window):
        # 시간별 예측값과 실제값 추출
        preds = ensemble_predictions[:, time_step, :]
        trues = true_values[:, time_step, :]
        
        # RMSE 계산
        mse = mean_squared_error(trues, preds, multioutput='uniform_average')
        rmse = np.sqrt(mse)
        ensemble_rmse_dict[time_step] = round(rmse, 4)

    # 전체 평균 RMSE 계산
    overall_rmse = np.mean(list(ensemble_rmse_dict.values()))

    return round(overall_rmse, 4)

In [63]:
import pandas as pd
import numpy as np
import os

def save_to_csv_for_seed(seed, monthly_rmse_list, monthly_ga_time_list, output_dir="./ga_검증"):
    """각 seed별로 CSV 파일로 저장하는 함수"""
    
    valid_months = [i for i in range(1, 12)]
    
    # 월별 RMSE와 GA 계산 시간을 DataFrame으로 변환
    monthly_df_rmse = pd.DataFrame({
        'Seed': [seed],
        **{f'Month {month} RMSE': [monthly_rmse_list[idx]] for idx, month in enumerate(valid_months)}
    })
    
    monthly_df_ga_time = pd.DataFrame({
        'Seed': [seed],
        **{f'Month {month} GA Time': [monthly_ga_time_list[idx]] for idx, month in enumerate(valid_months)}
    })
    
    # 저장할 디렉토리 생성 (없는 경우 생성)
    os.makedirs(output_dir, exist_ok=True)

    # 각 seed에 대해 결과 저장
    monthly_df_rmse.to_csv(f'{output_dir}/seed_{seed}_monthly_rmse.csv', index=False)
    monthly_df_ga_time.to_csv(f'{output_dir}/seed_{seed}_monthly_ga_time.csv', index=False)

    print(f"Seed {seed}의 CSV 파일들이 성공적으로 저장되었습니다.")



In [64]:
import numpy as np
import pandas as pd
import os

def calculate_seed_statistics(output_dir="./ga_검증"):
    """30개의 seed에 대한 전체 통계 (평균, 표준편차)를 계산하여 저장하는 함수"""
    all_rmse_list = []
    all_ga_time_list = []
    
    # 각 seed별 통계 계산을 위한 리스트
    seed_statistics = []

    for seed in range(31):  # 0부터 30까지 31번 반복 실험
        # seed별로 저장된 CSV 파일 불러오기
        monthly_df_rmse = pd.read_csv(f'{output_dir}/seed_{seed}_monthly_rmse.csv')
        monthly_df_ga_time = pd.read_csv(f'{output_dir}/seed_{seed}_monthly_ga_time.csv')
        
        # 전체 RMSE와 GA 시간의 값만 저장 (모든 월을 합친 통계)
        rmse_values = monthly_df_rmse.iloc[0, 1:].values  # 월별 RMSE
        ga_time_values = monthly_df_ga_time.iloc[0, 1:].values  # 월별 GA Time
        
        all_rmse_list.append(rmse_values)
        all_ga_time_list.append(ga_time_values)
        
        # 각 seed에 대한 전체 평균과 표준편차 계산
        seed_rmse_mean = np.mean(rmse_values)
        seed_ga_time_mean = np.mean(ga_time_values)
        
        # 각 seed별 통계를 저장
        seed_statistics.append({
            'Seed': seed,
            'Mean RMSE': seed_rmse_mean,
            'Mean GA Time': seed_ga_time_mean
        })
    
    # 전체 RMSE와 GA 시간에 대한 평균과 표준편차 계산
    total_rmse_mean = np.mean(all_rmse_list)
    total_ga_time_mean = np.mean(all_ga_time_list)

    # 최종 통계 DataFrame 생성 (전체 통계 + 각 seed별 통계)
    final_df = pd.DataFrame({
        'Total Mean RMSE': [total_rmse_mean],
        'Total Mean GA Time': [total_ga_time_mean],
    })

    # 각 seed별 통계를 DataFrame으로 변환
    seed_statistics_df = pd.DataFrame(seed_statistics)

    # 저장할 디렉토리 생성 (없는 경우 생성)
    os.makedirs(output_dir, exist_ok=True)

    # 전체 통계 결과를 CSV로 저장
    final_df.to_csv(f'{output_dir}/final_statistics.csv', index=False)
    # 각 seed별 통계 결과를 CSV로 저장
    seed_statistics_df.to_csv(f'{output_dir}/seed_statistics.csv', index=False)

    print(f"30개의 seed에 대한 통계 파일이 성공적으로 저장되었습니다.")


In [31]:
def validate_ga(input_directory, population_size=100, generations=100, mutation_rate=0.03, output_dir="./ga_검증"):
    """GA 가중치를 구하고, 테스트 데이터로 RMSE 계산"""
    overall_rmse_list = []
    ga_time_list = []

    for seed in range(31):  # 0부터 30까지 31번 반복 실험
        print(f"\nSeed {seed}로 실험 시작")
        
        np.random.seed(seed)  # 각 seed마다 난수 초기화
        
        monthly_rmse_list = []  # 매 `seed`마다 월별 RMSE 리스트 초기화
        monthly_ga_time_list = []  # 매 `seed`마다 월별 GA 계산 시간 리스트 초기화

        # 1월부터 12월까지 월별로 처리
        for month in range(1, 12):

            # 검증 데이터를 통해 GA 가중치 구하기
            model_predictions, true_values = load_and_process_validation_data(input_directory, month)  # 월별로 검증 데이터 로드
            optimal_weights, ga_time = run_ga_experiment(model_predictions, true_values, population_size, generations, mutation_rate, seed)
            
            # 테스트 데이터를 로드하여 최적 가중치로 예측 및 RMSE 계산
            test_predictions, test_true_values = load_and_process_test_data(input_directory, month)  # 월별로 테스트 데이터 로드
            test_rmse = calculate_ensemble_rmse2(test_predictions, test_true_values, optimal_weights)

            monthly_rmse_list.append(test_rmse)  # 월별 RMSE 추가
            monthly_ga_time_list.append(ga_time)  # GA 계산 시간도 리스트에 추가

        # 전체 실험 결과 (해당 seed에 대한 결과)
        overall_rmse_list.append(np.mean(monthly_rmse_list))  # 월별 RMSE 평균값을 overall에 추가
        ga_time_list.append(np.mean(monthly_ga_time_list))  # 월별 GA 계산 시간 평균값을 ga_time_list에 추가

        # 해당 seed에 대한 결과를 CSV로 저장
        save_to_csv_for_seed(seed, monthly_rmse_list.copy(), monthly_ga_time_list.copy(), output_dir)  # 복사된 리스트 전달

    # 모든 seed에 대해 통계 계산 후 CSV로 저장
    calculate_seed_statistics(output_dir)

    return overall_rmse_list, ga_time_list  # 결과 반환

In [32]:
# 실험에 사용할 입력 데이터 디렉토리 경로를 지정
input_directory = "./ga_w2"  # 데이터가 저장된 디렉토리 경로로 수정해주세요.
# validate_ga 함수 호출
# 실행 후 파일을 저장할 디렉토리 지정
output_dir = "./ga_검증_w2_2"  # 저장할 경로를 지정

# validate_ga 함수 실행하여 결과 가져오기
overall_rmse_list, ga_time_list, = validate_ga(input_directory, population_size=100, generations=100, mutation_rate=0.03, output_dir=output_dir)



Seed 0로 실험 시작
Generation 1/100, Best RMSE: 4.761268416323186
Generation 2/100, Best RMSE: 4.761268416323186
Generation 3/100, Best RMSE: 4.761268416323186
Generation 4/100, Best RMSE: 4.761268416323186
Generation 5/100, Best RMSE: 4.761268416323186
Generation 6/100, Best RMSE: 4.761268416323186
Generation 7/100, Best RMSE: 4.761268416323186
Generation 8/100, Best RMSE: 4.761268416323186
Generation 9/100, Best RMSE: 4.761268416323186
Generation 10/100, Best RMSE: 4.761268416323186
Generation 11/100, Best RMSE: 4.759915358486413
Generation 12/100, Best RMSE: 4.759915358486413
Generation 13/100, Best RMSE: 4.759915358486413
Generation 14/100, Best RMSE: 4.759915358486413
Generation 15/100, Best RMSE: 4.759915358486413
Generation 16/100, Best RMSE: 4.759915358486413
Generation 17/100, Best RMSE: 4.759915358486413
Generation 18/100, Best RMSE: 4.759915358486413
Generation 19/100, Best RMSE: 4.759915358486413
Generation 20/100, Best RMSE: 4.759915358486413
Generation 21/100, Best RMSE: 4.75

KeyboardInterrupt: 

In [274]:
print("\n최종 평균 RMSE")
print(f"전체 앙상블 평균 RMSE: {np.mean(overall_rmse_list)}")
print(f"GA 평균 계산 시간: {np.mean(ga_time_list)}")


최종 평균 RMSE
전체 앙상블 평균 RMSE: 3.142510850439882
GA 평균 계산 시간: 1.362541737095002
